<a href="https://colab.research.google.com/github/AprilWuWu/AI-Portfolio/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kaggle Api
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
# Kaggle Download
!kaggle competitions download -c titanic

  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 11.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.66MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 57.1MB/s]


In [ ]:
import pandas as pd
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [ ]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 合併
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
# axis (左右合併；上下合併)
data = pd.concat([train_df, test_df])

# drop axis=0 橫列； axis=1 直排
data = data.drop(['PassengerId', 'Survived'], axis=1)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
# Step1. 補缺失值

# 確認缺失多少
s = data.isna().sum()
s > 0
# 塞選 series[[True...]]
s[s > 0]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [ ]:
# Step1. 補缺失值

# 找最大值的索引
most = data['Embarked'].value_counts().idxmax()
data['Embarked'] = data['Embarked'].fillna(most)

In [ ]:
# 補數字類型[中位數] 基本款
med = data.median().drop(['Pclass'])
data = data.fillna(med)
data.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [ ]:
# 先準備我要保留的list
def name_cut(n):
  n = n.split(',')[-1].split('.')[0]
  return n.strip()

v = data['Name'].apply(name_cut).value_counts()
white = v.index[:4]

In [ ]:
def name_cut(n):
  n = n.split(',')[-1].split('.')[0]
  n = n.strip()
  if n in white:
    return n
  else: 
    return None
    # None 為 0, 0, 0 為不知道

data['Name'] = data['Name'].apply(name_cut)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,28.0,0,0,359309,8.0500,NaN,S


In [ ]:
v = data['Ticket'].value_counts()
def ticket(t):
  return v[t]

data['Ticket'] = data['Ticket'].apply(ticket) # 改成"多少人跟我有一樣的船票"

In [ ]:
data = data.drop(['Cabin'], axis=1)

In [ ]:
# 可以重置index
# data.reset_index(drop=True) 
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,28.0,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,28.0,0,0,1,8.0500,S


In [ ]:
# One-Hot Encoding
data = pd.get_dummies(data)

In [ ]:
# 取列: iloc(看實際是第幾列), loc(看列標籤)
# [第一列, 第二列, ...]
x_train = data.iloc[:len(train_df)]
x_test = data.iloc[len(train_df):]
y_train = train_df['Survived']

In [ ]:
# IterativeImputer 
# https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html

In [ ]:
data['Ticket'].value_counts()

1     713
2     264
3     147
4      64
7      35
5      35
6      24
8      16
11     11
Name: Ticket, dtype: int64

---
# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# random_state 可以取消隨機因素，隨便給一個整數

from sklearn.model_selection import GridSearchCV
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


# Grid Search

# clf = RandomForestClassifier(random_state=0)
# params = {
#     "n_estimators": range(20, 100),
#     "max_depth": range(5, 10)
# }
# search = GridSearchCV(clf, params, cv=10, n_jobs=-1)
# search.fit(x_train, y_train)
# print(search.best_params_)
# print(search.best_score_)


KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

clf = RandomForestClassifier(n_estimators=79, max_depth=6, random_state=0)
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores))

[0.81111111 0.87640449 0.76404494 0.8988764  0.86516854 0.82022472
 0.84269663 0.7752809  0.86516854 0.85393258]
0.8372908863920101


In [ ]:
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
df = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": pre
})
df.to_csv("titanic_rf.csv", encoding="utf-8", index=False)
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
# 參數重要程度
pd.DataFrame({
    "Name": data.columns,
    "Importance": clf.feature_importances_
})

,Name,Importance
0,Pclass,0.111763
1,Age,0.071098
2,SibSp,0.039382
3,Parch,0.018889
4,Ticket,0.071771
5,Fare,0.097300
6,Name_Master,0.018767
7,Name_Miss,0.046137
8,Name_Mr,0.123172
9,Name_Mrs,0.065655


In [ ]:
# !kaggle competitions submit -c titanic -f titanic_rf.csv -m "Message"

---
# KNN

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scale = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
x_train_scale = data_scale.iloc[:len(train_df)]
x_test_scale = data_scale.iloc[len(train_df):]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    "n_neighbors": range(5, 100),
}
search = GridSearchCV(clf, params, cv=10, n_jobs=-1)
search.fit(x_train_scale, y_train)
print(search.best_params_)
print(search.best_score_)

In [ ]:
clf = search.best_estimator_
pre = clf.predict(x_test_scale)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("titanic_knn.csv", encoding="utf-8", index=False)
df